In [ ]:
using Pkg
Pkg.activate(".")
#pkg"add CSV, DataFrames, NPZ, StatsBase, Random, Statistics, LaTeXStrings"

  Activating project at `~/Desktop/julia_codes/SmallCouplingDynamicCavity.jl/examples/SIS_inference`
   Resolving package versions...
    Updating `~/Desktop/julia_codes/SmallCouplingDynamicCavity.jl/examples/SIS_inference/Project.toml`
  [336ed68f] + CSV v0.10.15
⌅ [a93c6f00] + DataFrames v1.7.1
  [b964fa9f] + LaTeXStrings v1.4.0
  [15e1cf62] + NPZ v0.4.3
⌅ [2913bbd2] + StatsBase v0.34.4
  [9a3f8284] + Random
  [10745b16] + Statistics v1.9.0
    Updating `~/Desktop/julia_codes/SmallCouplingDynamicCavity.jl/examples/SIS_inference/Manifest.toml`
  [66dad0bd] + AliasTables v1.1.3
  [336ed68f] + CSV v0.10.15
  [944b1d66] + CodecZlib v0.7.8
  [a8cc5b0e] + Crayons v4.1.1
  [9a962f9c] + DataAPI v1.16.0
⌅ [a93c6f00] + DataFrames v1.7.1
  [e2d170a0] + DataValueInterfaces v1.0.0
  [ffbed154] + DocStringExtensions v0.9.5
  [48062228] + FilePathsBase v0.9.24
  [842dd82b] + InlineStrings v1.4.5
  [41ab1584] + InvertedIndices v1.3.1
⌅ [92d709cd] + IrrationalConstants v0.2.4
  [82899510] + IteratorI

In [12]:
using Graphs
using JLD2, CSV, DataFrames, NPZ
using StatsBase
using PyCall
using Random
import PyPlot as plt
using Statistics
using LaTeXStrings

In [13]:
# Define the saving structure, in which the results will be saved
struct SaveStruct
    s::Int
    ε_autoinf::Float64
    maxiter::Union{Int,Array{Int}}
    damp::Union{Float64,Array{Float64}}
    μ_cutoff::Float64
    fpn::Float64
    n_iter_nc::Int
    damp_nc::Float64
    fMAP_true::Float64

    SaveStruct(s) = new(s, 0.0, 0, 0.0, 0.0, 0.0, 0, 0.0, 0.0)
    SaveStruct(s, ε_autoinf, maxiter, damp, μ_cutoff, fpn, n_iter_nc, damp_nc, fMAP_true) = new(s, ε_autoinf, maxiter, damp, μ_cutoff, fpn, n_iter_nc, damp_nc, fMAP_true)
end

to_rgba(col, alpha) = plt.matplotlib.colors.to_rgba(col, alpha)

function meshgrid_lin(x, y, dx, dy)
    nx = length(x)+1
    ny = length(y)+1
    X = zeros(ny,nx)
    Y = zeros(ny,nx)
    for j in 1:nx-1
        X[:,j] .= x[j]-dx/2
    end
    X[:,nx] .= x[nx-1]+dx/2
    
    for i in 1:ny-1
        Y[i,:] .= y[i]-dy/2
    end
    Y[ny,:] .= y[ny-1]+dy/2
    return X, Y
end

meshgrid_lin (generic function with 1 method)

# Varying lambda, sigma

## Proximity random graphs

In [16]:
n_sample = 30
sims = collect(1:n_sample)

# Parameters
T = 15
NV = 300
lmax = round(sqrt(1.8/NV), digits=3)
n_sr = 2
γ = n_sr/NV
λs = round.(collect(range(0.1, 0.9, length=20)), digits=3)
σs = round.(10 .^ (collect(range(log10(0.01), log10(0.9), length=20))), digits=3)
λ_σs = collect(Iterators.product(λs, σs))
nobs = floor(Int, .6*NV)

tobs = floor(Int, T/2)

############### SCDC #############################
JLD2.@load "SIS_dataplot_lambda_sigma.jld2"
##################################################

################## Plot fMAP_true  ####################
fig, ax = plt.subplots(figsize=(3.4, 2))
rcParams = PyDict(plt.matplotlib["rcParams"])
rcParams["font.size"] = 8  #9 18

# Custom color gradient
color1 = "dodgerblue"  # Starting color
color2 = "yellow"      # Ending color
# Create a colormap using LinearSegmentedColormap
cm = plt.matplotlib.colors.LinearSegmentedColormap.from_list("custom_cmap", [color1, color2])

## Heatmap ##
# Define axes
y_edges = λs  # Edges for x-axis (0.5 to NV + 0.5)
dy = y_edges[2] - y_edges[1]
x_edges_exp = collect(range(log10(0.01), log10(0.9), length=20)) # Edges for y-axis (0 to T)
dx = x_edges_exp[2] - x_edges_exp[1]
# Create X, Y coordinate matrices
X, Y = meshgrid_lin(x_edges_exp, y_edges, dx, dy)
# Plot heatmap
p = ax.pcolor(10 .^ X, Y, fMAP_trues_m, cmap=cm, vmin=minimum(fMAP_trues_m),  vmax=1, edgecolors="none", shading="flat")
ax.set_ylabel(L"$\lambda$") 
ax.set_xlabel(L"$\sigma$")
ax.set_ylim(y_edges[1]-dy/2, y_edges[end]+dy/2)
ax.set_yticks(.1:.2:.9)
ax.set_xlim(10 .^ (x_edges_exp[1]-dx/2), 10 .^ (x_edges_exp[end]+dx/2))
ax.set_xscale("log")

#ax.set_yscale("log")
####################################################

# Add colorbars with adjusted width and position
cbar = fig.colorbar(p, ax=ax, orientation="vertical", fraction=0.1, pad=.02, aspect=10, drawedges=false)
cbar.set_label("Fraction of correct\n MAP predictions")

fig.savefig("SIS_heatmap_inference.png", format="png", bbox_inches="tight", dpi=2000)

PyCall.PyError: PyError ($(Expr(:escape, :(ccall(#= /Users/mattiatarabolo/.julia/packages/PyCall/1gn3u/src/pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'TypeError'>
TypeError("Dimensions of C (10, 10) should be one smaller than X(21) and Y(21) while using shading='flat' see help(pcolor)")
  File "/Users/mattiatarabolo/.julia/conda/3/aarch64/lib/python3.12/site-packages/matplotlib/__init__.py", line 1521, in inner
    return func(
           ^^^^^
  File "/Users/mattiatarabolo/.julia/conda/3/aarch64/lib/python3.12/site-packages/matplotlib/axes/_axes.py", line 6259, in pcolor
    X, Y, C, shading = self._pcolorargs('pcolor', *args, shading=shading,
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mattiatarabolo/.julia/conda/3/aarch64/lib/python3.12/site-packages/matplotlib/axes/_axes.py", line 6054, in _pcolorargs
    raise TypeError(f"Dimensions of C {C.shape} should"
